# Testing Race plot

In [1]:
import pandas as pd
from datetime import date
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import os
import datetime

In [2]:
path = "\\".join(os.getcwd().split("\\")[:-2]) + "\\common_data"
path

'C:\\Users\\Camila\\Documents\\git\\common_data'

In [28]:
# Importing the Excel file with all the cities
covid_munic = pd.read_csv(path+"\data\caso_full.csv")
covid_munic["date"] = pd.to_datetime(covid_munic['date'], format="%Y/%m/%d")
covid_munic["date_formatted"] = covid_munic["date"].apply(lambda x: x.strftime('%d/%m'))

region = {'AC': "Norte", 'AM': "Norte", 'AP': "Norte", 'PA': "Norte", 'RO': "Norte", 'RR': "Norte", 'TO': "Norte",
          'AL': "Nordeste", 'BA': "Nordeste", 'CE': "Nordeste", 'MA': "Nordeste", 'PB': "Nordeste", 'PE': "Nordeste", 
          'PI': "Nordeste", 'RN': "Nordeste", 'SE': "Nordeste", 
          'DF': "Centro Oeste", 'GO': "Centro Oeste", 'MS': "Centro Oeste", 'MT': "Centro Oeste",
          'ES': "Sudeste", 'MG': "Sudeste", 'RJ': "Sudeste", 'SP': "Sudeste",
          'PR': "Sul", 'SC': "Sul", 'RS': "Sul"}

covid_munic['Region'] = covid_munic['state'].map(region)

covid_munic['Death Rate'] = covid_munic['last_available_deaths'] / covid_munic['last_available_confirmed'] * 100

covid_munic.head(1)

,epidemiological_week,date,order_for_place,state,city,city_ibge_code,place_type,last_available_confirmed,last_available_confirmed_per_100k_inhabitants,new_confirmed,last_available_deaths,new_deaths,last_available_death_rate,estimated_population_2019,is_last,is_repeated,date_formatted,Region,Death Rate
0,22,2020-05-27,60,AC,Acrelândia,1200013.0,city,129,845.56896,0,1,0,0.0078,15256.0,False,True,27/05,Norte,0.775194


In [23]:
covid_uf = covid_munic[covid_munic["place_type"] == "state"].groupby(["state", "date"])["last_available_confirmed", "last_available_deaths", "new_confirmed", "new_deaths"].sum().reset_index()
covid_uf["date"] = pd.to_datetime(covid_uf['date'], format="%Y/%m/%d")
covid_uf["date_formatted"] = covid_uf["date"].apply(lambda x: x.strftime('%d/%m'))

# Create Region
region = {'AC': "Norte", 'AM': "Norte", 'AP': "Norte", 'PA': "Norte", 'RO': "Norte", 'RR': "Norte", 'TO': "Norte",
          'AL': "Nordeste", 'BA': "Nordeste", 'CE': "Nordeste", 'MA': "Nordeste", 'PB': "Nordeste", 'PE': "Nordeste", 
          'PI': "Nordeste", 'RN': "Nordeste", 'SE': "Nordeste", 
          'DF': "Centro Oeste", 'GO': "Centro Oeste", 'MS': "Centro Oeste", 'MT': "Centro Oeste",
          'ES': "Sudeste", 'MG': "Sudeste", 'RJ': "Sudeste", 'SP': "Sudeste",
          'PR': "Sul", 'SC': "Sul", 'RS': "Sul"}

covid_uf['Region'] = covid_uf['state'].map(region)

covid_uf['Death Rate'] = covid_uf['last_available_deaths'] / covid_uf['last_available_confirmed'] * 100

covid_uf.tail()

C:\Users\Camila\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,state,date,last_available_confirmed,last_available_deaths,new_confirmed,new_deaths,date_formatted,Region,Death Rate
2051,TO,2020-05-23,2430,51,0,0,23/05,Norte,2.098765
2052,TO,2020-05-24,2430,51,0,0,24/05,Norte,2.098765
2053,TO,2020-05-25,2696,58,266,7,25/05,Norte,2.151335
2054,TO,2020-05-26,2858,64,162,6,26/05,Norte,2.239328
2055,TO,2020-05-27,2858,64,0,0,27/05,Norte,2.239328


In [11]:
covid_uf[(covid_uf["new_confirmed"] > 0) & (covid_uf["new_deaths"] > 0)].groupby("date")["state"].count().tail(10)

date
2020-05-18    27
2020-05-19    24
2020-05-20    25
2020-05-21    24
2020-05-22    26
2020-05-23    20
2020-05-24    23
2020-05-25    24
2020-05-26    22
2020-05-27    14
Name: state, dtype: int64

In [17]:
confirmed = covid_uf.pivot_table(index=["state","Region"], columns=["date", "date_formatted"], aggfunc={'last_available_confirmed':sum}, fill_value=0).reset_index()

In [20]:
confirmed.to_csv('confirmed_racebars.csv', encoding='utf-8', index=False)

In [21]:
death = covid_uf.pivot_table(index=["state","Region"], columns=["date", "date_formatted"], aggfunc={'last_available_deaths':sum}, fill_value=0).reset_index()
death.to_csv('deaths_racebars.csv', encoding='utf-8', index=False)


In [25]:
death_rate = covid_uf.pivot_table(index=["state","Region"], columns=["date", "date_formatted"], aggfunc={'Death Rate':sum}, fill_value=0).reset_index()
death_rate.to_csv('death_rate_racebars.csv', encoding='utf-8', index=False)

In [32]:
confirmed_munic = covid_munic[covid_munic["place_type"] == "city"].pivot_table(index=["city_ibge_code","city","Region"], columns=["date", "date_formatted"], aggfunc={'last_available_confirmed':max}, fill_value=0).reset_index()
confirmed_munic.to_csv('confirmed_munic_racebars.csv', encoding='utf-8', index=False)


In [34]:
death_munic = covid_munic[covid_munic["place_type"] == "city"].pivot_table(index=["city_ibge_code","city","Region"], columns=["date", "date_formatted"], aggfunc={'last_available_deaths':max}, fill_value=0).reset_index()
death_munic.to_csv('deaths_munic_racebars.csv', encoding='utf-8', index=False)
